# Speedy Python

Some examples how to measure speed and memory usage.

Comparisons include how to speed up or replace loops with built-in functions or NumPy arrays.

## Timeit
**Line Magic**

In [1]:
# default params
%timeit lambda: "-".join(map(str, range(10000)))

46.8 ns ± 2.66 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [2]:
# custom params
%timeit -r 10 -n 1000 lambda: "-".join(map(str, range(10000)))

40.6 ns ± 0.135 ns per loop (mean ± std. dev. of 10 runs, 1000 loops each)


**Cell magic**

Note: the magic command has to be in the first line of the cell or it won't work

In [3]:
%%timeit
total = 0
for i in range(100):
    for j in range(100):
        total += i * (-1) ** j

3.34 ms ± 52.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Example Setup

In [4]:
import random
import string
import numpy as np

# a function that generates a random numeric and alphabetical an numeric string
def randStr(chars = string.ascii_lowercase, N=10):
    return ''.join(random.choice(chars) for _ in range(N)).title()

def randNamegen(count=1000, length=10):
    nameList=[]
    for i in range(count):
        nameList.append(randStr(N=length))
    return nameList

nameList = randNamegen(count=100000)
print("List",nameList[:5])

nameList_np = np.array(nameList)
print("NumPy Array",nameList_np[:5])

List ['Xleoufudlp', 'Jspyzryebq', 'Bairgjmpei', 'Lwnrxwaanp', 'Angmhrqtkn']
NumPy Array ['Xleoufudlp' 'Jspyzryebq' 'Bairgjmpei' 'Lwnrxwaanp' 'Angmhrqtkn']


In [5]:
# a function that generates randomized people's heights in centimeters
def randHTgen(count=1000, lower=150, upper=300):
    HTList=[]
    for i in range(count):
        HTList.append(random.randint(lower, upper))
    return HTList

HTList = randHTgen(count=100000)
print("List",HTList[:5])

HTList_np = np.array(HTList)
print("NumPy Array",HTList_np[:5])

List [251, 198, 172, 291, 252]
NumPy Array [251 198 172 291 252]


In [6]:
# a function that generates a range of random people wights in kilogramm
def randWTgen(count=1000, lower=50, upper=150):
    WTList=[]
    for i in range(count):
        WTList.append(random.randint(lower, upper))
    return WTList

WTList = randWTgen(count=100000)
print("List",WTList[:5])

WTList_np = np.array(WTList)
print("NumPy Array",WTList_np[:5])

List [74, 105, 54, 53, 79]
NumPy Array [ 74 105  54  53  79]


**The example function converts the heights from centimeter to feet and weights from kilograms to pounds.**

In [7]:
# a function for measuring (list comprehension)
def convert_units_list(names, heights, weights):
    new_hts = [ht * 0.39370  for ht in heights]
    new_wts = [wt * 2.20462  for wt in weights]
    people_data = {}
    for i,name in enumerate(names):
        people_data[name] = (new_hts[i], new_wts[i])
    return people_data

In [8]:
# a function for measuring (NumPy array broadcasting)
def convert_units_array(names, heights, weights):
    new_hts = heights * 0.39370
    new_wts = weights * 2.20462
    people_data = {}
    for i,name in enumerate(names):
        people_data[name] = (new_hts[i], new_wts[i])
    return people_data

#convert_units_array(nameList_np, HTList_np, WTList_np)

## Case Study: list vs NumPy array
### Line Profiler

In [9]:
%load_ext line_profiler

%lprun -f convert_units_list convert_units_list(nameList, HTList, WTList)

Timer unit: 1e-07 s

Total time: 0.213249 s
File: <ipython-input-7-b143edde93f5>
Function: convert_units_list at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def convert_units_list(names, heights, weights):
     3         1     184181.0 184181.0      8.6      new_hts = [ht * 0.39370  for ht in heights]
     4         1     171217.0 171217.0      8.0      new_wts = [wt * 2.20462  for wt in weights]
     5         1         21.0     21.0      0.0      people_data = {}
     6    100001     883878.0      8.8     41.4      for i,name in enumerate(names):
     7    100000     893188.0      8.9     41.9          people_data[name] = (new_hts[i], new_wts[i])
     8         1          7.0      7.0      0.0      return people_data

In [10]:
%lprun -f convert_units_array convert_units_array(nameList_np, HTList_np, WTList_np)

Timer unit: 1e-07 s

Total time: 0.165588 s
File: <ipython-input-8-5226aaf1be87>
Function: convert_units_array at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def convert_units_array(names, heights, weights):
     3         1       5837.0   5837.0      0.4      new_hts = heights * 0.39370
     4         1       5605.0   5605.0      0.3      new_wts = weights * 2.20462
     5         1         15.0     15.0      0.0      people_data = {}
     6    100001     813374.0      8.1     49.1      for i,name in enumerate(names):
     7    100000     831042.0      8.3     50.2          people_data[name] = (new_hts[i], new_wts[i])
     8         1          6.0      6.0      0.0      return people_data

### Memory Profiler

In [11]:
from conv_list import convert_units_list

%load_ext memory_profiler

%mprun -f convert_units_list convert_units_list(nameList, HTList, WTList)

Filename: C:\Users\ChristianV700\Documents\GitHub\Python_coding\speedy_python\conv_list.py

Line #    Mem usage    Increment  Occurences   Line Contents
     1     76.8 MiB     76.8 MiB           1   def convert_units_list(names, heights, weights):
     2     81.1 MiB      4.3 MiB      100003       new_hts = [ht * 0.39370  for ht in heights]
     3     85.7 MiB      4.6 MiB      100003       new_wts = [wt * 2.20462  for wt in weights]
     4     85.7 MiB      0.0 MiB           1       people_data = {}
     5     96.9 MiB      6.1 MiB      100001       for i,name in enumerate(names):
     6     96.9 MiB      5.0 MiB      100000           people_data[name] = (new_hts[i], new_wts[i])
     7     96.9 MiB      0.0 MiB           1       return people_data

In [12]:
from conv_array import convert_units_array

%mprun -f convert_units_array convert_units_array(nameList_np, HTList_np, WTList_np)

Filename: C:\Users\ChristianV700\Documents\GitHub\Python_coding\speedy_python\conv_array.py

Line #    Mem usage    Increment  Occurences   Line Contents
     1     77.4 MiB     77.4 MiB           1   def convert_units_array(names, heights, weights):
     2     78.3 MiB      0.8 MiB           1       new_hts = heights * 0.39370
     3     79.0 MiB      0.8 MiB           1       new_wts = weights * 2.20462
     4     79.0 MiB      0.0 MiB           1       people_data = {}
     5    107.4 MiB     22.1 MiB      100001       for i,name in enumerate(names):
     6    107.4 MiB      6.2 MiB      100000           people_data[name] = (new_hts[i], new_wts[i])
     7    107.4 MiB      0.0 MiB           1       return people_data

## Combining items of two lists (loop vs zip)

Let's use the randomized names list and the randomized heights list.

In [13]:
combined = []
for i, name in enumerate(nameList):
    combined.append((name, HTList[i]))
print(combined[:5])

[('Xleoufudlp', 251), ('Jspyzryebq', 198), ('Bairgjmpei', 172), ('Lwnrxwaanp', 291), ('Angmhrqtkn', 252)]


In [14]:
%%timeit
combined = []
for i, name in enumerate(nameList):
    combined.append((name, HTList[i]))

17.8 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
combined = []
for i, name in enumerate(nameList):
    combined.append((name, HTList[i]))
# unpack zip object to list
unpacked = [*combined]
print(unpacked[:5])

[('Xleoufudlp', 251), ('Jspyzryebq', 198), ('Bairgjmpei', 172), ('Lwnrxwaanp', 291), ('Angmhrqtkn', 252)]


In [16]:
%%timeit
combined_zip = zip(nameList, HTList)

194 ns ± 1.06 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Counting items on a list (loop vs Counter function)

In [17]:
height_counts = {}
for height in HTList:
    if height not in height_counts:
        height_counts[height] = 1
    else:
        height_counts[height] += 1
print(height_counts)

{251: 694, 198: 701, 172: 703, 291: 687, 252: 631, 159: 649, 249: 668, 168: 678, 234: 669, 188: 641, 243: 670, 255: 637, 196: 657, 152: 702, 296: 679, 204: 708, 158: 614, 283: 599, 213: 652, 162: 644, 222: 672, 281: 642, 274: 625, 297: 665, 292: 691, 238: 686, 236: 634, 240: 707, 161: 654, 199: 671, 150: 696, 154: 704, 156: 651, 174: 637, 187: 671, 180: 665, 177: 712, 275: 696, 202: 653, 219: 656, 239: 684, 260: 645, 200: 647, 286: 640, 261: 649, 237: 697, 203: 677, 190: 666, 288: 679, 269: 677, 214: 633, 170: 682, 184: 676, 229: 641, 298: 605, 166: 715, 233: 639, 189: 609, 276: 625, 169: 655, 185: 644, 259: 626, 220: 611, 295: 673, 287: 684, 221: 644, 248: 669, 264: 691, 224: 608, 175: 687, 290: 622, 212: 673, 211: 646, 210: 660, 157: 695, 225: 718, 247: 681, 258: 689, 256: 680, 293: 725, 164: 704, 280: 674, 272: 625, 160: 679, 241: 657, 299: 655, 250: 716, 278: 656, 284: 678, 245: 674, 289: 653, 242: 659, 300: 661, 217: 659, 171: 677, 163: 666, 266: 672, 167: 634, 235: 650, 285: 655,

In [18]:
%%timeit
height_counts = {}
for height in HTList:
    if height not in height_counts:
        height_counts[height] = 1
    else:
        height_counts[height] += 1

12.4 ms ± 382 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
from collections import Counter
height_counts = Counter(HTList)
print(height_counts)

Counter({293: 725, 244: 719, 225: 718, 265: 717, 250: 716, 166: 715, 177: 712, 204: 708, 240: 707, 154: 704, 164: 704, 172: 703, 271: 703, 152: 702, 198: 701, 277: 701, 237: 697, 232: 697, 150: 696, 275: 696, 157: 695, 251: 694, 292: 691, 264: 691, 267: 690, 258: 689, 226: 689, 205: 689, 257: 688, 291: 687, 175: 687, 238: 686, 183: 685, 263: 685, 239: 684, 287: 684, 170: 682, 247: 681, 256: 680, 296: 679, 288: 679, 160: 679, 168: 678, 284: 678, 203: 677, 269: 677, 171: 677, 155: 677, 184: 676, 280: 674, 245: 674, 197: 674, 295: 673, 212: 673, 176: 673, 222: 672, 266: 672, 199: 671, 187: 671, 243: 670, 227: 670, 234: 669, 248: 669, 215: 669, 228: 669, 279: 669, 249: 668, 190: 666, 163: 666, 182: 666, 297: 665, 180: 665, 173: 665, 195: 662, 300: 661, 230: 661, 210: 660, 246: 660, 242: 659, 217: 659, 262: 659, 208: 658, 201: 658, 196: 657, 241: 657, 219: 656, 278: 656, 169: 655, 299: 655, 285: 655, 161: 654, 193: 654, 202: 653, 289: 653, 213: 652, 156: 651, 235: 650, 165: 650, 186: 650, 1

In [20]:
%%timeit
height_counts = Counter(HTList)

4.96 ms ± 26.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Number of combinations on a list (loop vs Combinations function)

In [21]:
pairs = []
for x in nameList[:100]:
    for y in nameList[:100]:
        if x == y:
            continue
        if ((x,y) not in pairs) & ((y,x) not in pairs):
            pairs.append((x,y))
print(pairs[:5])

[('Xleoufudlp', 'Jspyzryebq'), ('Xleoufudlp', 'Bairgjmpei'), ('Xleoufudlp', 'Lwnrxwaanp'), ('Xleoufudlp', 'Angmhrqtkn'), ('Xleoufudlp', 'Bpiejoqtly')]


In [22]:
%%timeit
for x in nameList[:100]:
    for y in nameList[:100]:
        if x == y:
            continue
        if ((x,y) not in pairs) & ((y,x) not in pairs):
            pairs.append((x,y))

1.98 s ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
from itertools import combinations

pairs_obj = combinations(nameList[:100],2)

# unpack combinations object to list
pairs = [*pairs_obj]
print(pairs[:5])

[('Xleoufudlp', 'Jspyzryebq'), ('Xleoufudlp', 'Bairgjmpei'), ('Xleoufudlp', 'Lwnrxwaanp'), ('Xleoufudlp', 'Angmhrqtkn'), ('Xleoufudlp', 'Bpiejoqtly')]


In [24]:
%%timeit
combinations(nameList[:100],2)

724 ns ± 6.85 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Replace loops with built-in functions (zip, map)

In [25]:
loop_output = []
for name,weight in zip(nameList, WTList):
    if weight < 100:
        name_length = len(name)
        tuple = (name, name_length)
        loop_output.append(tuple)

loop_output[:5]

[('Xleoufudlp', 10),
 ('Bairgjmpei', 10),
 ('Lwnrxwaanp', 10),
 ('Angmhrqtkn', 10),
 ('Usespvmygc', 10)]

In [26]:
%%timeit
loop_output = []
for name,weight in zip(nameList, WTList):
    if weight < 100:
        name_length = len(name)
        tuple = (name, name_length)
        loop_output.append(tuple)

11.7 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
# use list comprehension to generate a filtered new list
filtered_name_list = [name for name,weight in zip(nameList, WTList) if weight > 100]

# use map() to apply a function to a list
name_lengths_map = map(len, filtered_name_list)

# Combine two lists with zip, then unpack zip
output = [*zip(filtered_name_list, name_lengths_map)]

output[:5]

[('Jspyzryebq', 10),
 ('Bpiejoqtly', 10),
 ('Ikslqgziit', 10),
 ('Ebphiyxefz', 10),
 ('Hwabfjbpek', 10)]

In [28]:
%%timeit
filtered_name_list = [name for name,weight in zip(nameList, WTList) if weight > 100]
name_lengths_map = map(len, filtered_name_list)
output = [*zip(filtered_name_list, name_lengths_map)]

8.95 ms ± 280 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
